# Example bulk data client interaction

Implements a simple [FHIR bulk data client](http://wiki.hl7.org/index.php?title=201801_Bulk_Data) that invokes the `$export` operation and converts the resulting data stream to a generator.

### Config

Start by reading required config parameters.

In [1]:
import yaml

with open('config.yaml') as f:
    config = yaml.load(f)

### Setup

Construct the client. First we request the remote server to prepare a dataset using ``client.provision()``

We can check for successful provisioning of the list of data files using a property of the client

In [2]:
from fhir.client import BulkDataClient

client = BulkDataClient(**config)
client.provision(_type='Patient')

In [3]:
client.provisioned

True

In [4]:
client.manifest

['https://s3.us-south.cloud-object-storage.appdomain.cloud/fhir-r4-connectathon/job74/Patient_0.ndjson']

### Consume data

Once the dataset is provisioned we can iterate through the resources in the bulk data set. The client leaves most data  on the remote server and waits to retrieve subsequent files until the previous file in the manifest is consumed.

In [5]:
data = client.iter_json()
next(data)

{'resourceType': 'Patient',
 'id': 'db049c87-0b30-447f-b995-3d700b61f36d',
 'meta': {'versionId': '1', 'lastUpdated': '2019-09-13T19:26:19.6Z'},
 'active': True,
 'name': [{'family': 'Ortiz', 'given': ['David']}],
 'gender': 'male'}

### Cleanup

We should close the connection in the underlying session prior to releasing the client. Alternatively the client also functions as a context manager for simplicity.

In [6]:
client.session.close()

with BulkDataClient(**config) as newclient:
    newclient.provision(_type='Patient')
    print(newclient.manifest)

['https://s3.us-south.cloud-object-storage.appdomain.cloud/fhir-r4-connectathon/job75/Patient_0.ndjson']
